In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import time
import plotly
from datetime import datetime
import optuna
import sklearn
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
SEED = 10
limit_rows = None

In [3]:
print(f"Execution started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

Execution started at 2022-03-02 04:32:16.


In [4]:
def load_csv(name):
    if limit_rows is None:
        full_filename = f'../Data Preprocessing/sklearn/full/iot23_combined_{name}.csv'
    else:
        full_filename = f'../Data Preprocessing/sklearn/semi/iot23_combined_{int(limit_rows/1000)}k_{name}.csv'
    
    df = pd.read_table(filepath_or_buffer=full_filename, header=None, sep=',').infer_objects().to_numpy()
    
    return df.ravel() if df.shape[1] == 1 else df

In [5]:
X_train, X_test, y_train, y_test = load_csv('X_train'), load_csv('X_test'), load_csv('y_train'), load_csv('y_test')

print('X_train',X_train.shape,'\ny_train',y_train.shape)
print('X_test',X_test.shape,'\ny_test',y_test.shape)

X_train (7142855, 15) 
y_train (7142855,)
X_test (1785714, 15) 
y_test (1785714,)


In [6]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

n_features = X_train.shape[1]
n_classes = len(set(y_train))

def objective(trial):
    
    solver       = trial.suggest_categorical('solver', ['svd', 'lsqr', 'eigen'])
    shrinkage    = trial.suggest_categorical('shrinkage', [None, 'auto'])
    n_components = trial.suggest_int('n_components', 1, min(n_features, n_classes-1))

    classifier_obj = LinearDiscriminantAnalysis(solver=solver,
                                                shrinkage=shrinkage,
                                                n_components=n_components)
    
    classifier_obj.fit(X_train, y_train)
    
    y_pred = classifier_obj.predict(X_test)
    
    accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
    
    return accuracy

In [7]:
print(f"Optimization started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

Optimization started at 2022-03-02 04:32:28.


In [ ]:
study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=100, n_jobs=4, catch=(NotImplementedError,))

[I 2022-03-02 04:32:28,504] A new study created in memory with name: no-name-4bf87453-df2b-43cf-9bba-bb704abaea83
C:\Users\marce\anaconda3\envs\tf\lib\site-packages\optuna\study\study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[W 2022-03-02 04:32:30,065] Trial 2 failed because of the following error: NotImplementedError('shrinkage not supported')
Traceback (most recent call last):
  File "C:\Users\marce\anaconda3\envs\tf\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\marce\AppData\Local\Temp/ipykernel_5116/800447189.py", line 16, in objective
    classifier_obj.fit(X_train, y_train)
  File "C:\Users\marce\anaconda3\envs\tf\lib\site-packages\sklearn\discriminant_analysis.py", line 583, in fit
    raise NotImplementedError("shrinkage not supported")
NotImplementedError: shr

[I 2022-03-02 04:34:37,832] Trial 33 finished with value: 0.7072537931606069 and parameters: {'solver': 'lsqr', 'shrinkage': None, 'n_components': 8}. Best is trial 0 with value: 0.7072537931606069.
[I 2022-03-02 04:34:39,546] Trial 34 finished with value: 0.7072537931606069 and parameters: {'solver': 'eigen', 'shrinkage': None, 'n_components': 8}. Best is trial 0 with value: 0.7072537931606069.
[I 2022-03-02 04:34:41,875] Trial 35 finished with value: 0.7072537931606069 and parameters: {'solver': 'eigen', 'shrinkage': None, 'n_components': 12}. Best is trial 0 with value: 0.7072537931606069.
[I 2022-03-02 04:34:46,718] Trial 36 finished with value: 0.7072537931606069 and parameters: {'solver': 'eigen', 'shrinkage': None, 'n_components': 13}. Best is trial 0 with value: 0.7072537931606069.
[I 2022-03-02 04:34:50,724] Trial 37 finished with value: 0.7072537931606069 and parameters: {'solver': 'eigen', 'shrinkage': None, 'n_components': 13}. Best is trial 0 with value: 0.7072537931606069

[I 2022-03-02 04:37:00,007] Trial 72 finished with value: 0.7072537931606069 and parameters: {'solver': 'eigen', 'shrinkage': None, 'n_components': 14}. Best is trial 0 with value: 0.7072537931606069.
[I 2022-03-02 04:37:04,258] Trial 73 finished with value: 0.7072537931606069 and parameters: {'solver': 'eigen', 'shrinkage': None, 'n_components': 14}. Best is trial 0 with value: 0.7072537931606069.
[I 2022-03-02 04:37:07,119] Trial 74 finished with value: 0.7072537931606069 and parameters: {'solver': 'eigen', 'shrinkage': None, 'n_components': 6}. Best is trial 0 with value: 0.7072537931606069.
[I 2022-03-02 04:37:11,332] Trial 75 finished with value: 0.7072537931606069 and parameters: {'solver': 'eigen', 'shrinkage': None, 'n_components': 15}. Best is trial 0 with value: 0.7072537931606069.
[I 2022-03-02 04:37:12,722] Trial 76 finished with value: 0.7072537931606069 and parameters: {'solver': 'lsqr', 'shrinkage': None, 'n_components': 9}. Best is trial 0 with value: 0.7072537931606069

In [ ]:
print(f"Optimization finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

In [ ]:
def format_trial(trial):
    return json.dumps({'number' : trial.number,
                       'score'  : trial.values[0],
                       'params' : trial.params}, indent=4, default=str)

In [ ]:
print(format_trial(study.best_trial))

In [ ]:
for trial in study.best_trials:
    print(format_trial(trial))

In [ ]:
optuna.visualization.plot_optimization_history(study).show()